### Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime, timedelta
from IPython.core.display import display, HTML, clear_output
from ipywidgets import Button, Text, Layout, Box, Output
import functools

### Выгрузка данных
Cсылка на API: https://github.com/hhru/api 

In [2]:
# Выбор названий, временного отрезка и опыта

text_list = [
            'Junior Data Scientist',
            'Data Scientist',
            'Junior Data Science',
            'Data Science',
            'Джуниор Дата Саентист',
            'Дата Cаентист',
            'Machine Learning',
            'Машинное обучение',
            'Специалист по machine learning',
            'Специалист по машинному обучению',
            'Machine Learning Engineer',
            'Machine Learning Developer',
            'Machine Learning Researcher',
            'Разработчик Machine Learning',
            'Machine Learning инженер',
            'ML Engineer',
            'ML Developer',
            'ML Researcher',
            'Разработчик ML',
            'ML инженер',
#             'Data Analyst',
#             'Analyst',
#             'Аналитик данных',
#             'Специалист по анализу данных',
#             'Аналитик'
            ]

text_str = '" or "'.join([i for i in text_list])
text = f'SQL NAME:("{text_str}")'

dt_now = datetime.now().date()
dt_from = dt_now - timedelta(days=1)
date_to = '%s-%s-%s' % (dt_now.year, dt_now.month, dt_now.day)
date_from = '%s-%s-%s' % (dt_from.year, dt_from.month, dt_from.day)

url_0 = 'https://api.hh.ru/'
url_1 = url_0 + f'vacancies?text={text}&date_from={date_from}&date_to={date_to}&responses_count_enabled=True&per_page=100'

experiences = ['noExperience', 'between1And3']

In [3]:
# Выгрузка данных

request_data = pd.DataFrame()
for exp in experiences:
    page = 0
    url_2 = url_1 + f'&experience={exp}'
    try:
        r = requests.get(url_2 + f'&page={page}')
        data = json.loads(r.text)
        print(exp, ' - ', data['pages'], 'pages')
        items = data['items']
        for page in range(1, data['pages']):
            r = requests.get(url_2 + f'&page={page}')
            data = json.loads(r.text)
            items += data['items']
        df = pd.DataFrame(items)
        df['experience'] = exp
        request_data = pd.concat([request_data, df])
    except Exception as e:
        print(exp, '-', e)
        
request_data.reset_index(drop=True, inplace=True)
print('\nDataFrame shape:', request_data.shape)
pd.set_option('display.max_columns', None);
request_data.sample(3)

noExperience  -  2 pages
between1And3  -  7 pages

DataFrame shape: (798, 30)


,id,premium,name,department,has_test,response_letter_required,area,salary,type,address,response_url,sort_point_distance,published_at,created_at,archived,apply_alternate_url,insider_interview,url,alternate_url,relations,employer,snippet,contacts,schedule,counters,working_days,working_time_intervals,working_time_modes,accept_temporary,experience
4,48710966,False,Аналитик-стажер,None,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}","{'city': 'Москва', 'street': 'улица Земляной В...",None,None,2021-10-29T09:51:39+0300,2021-10-29T09:51:39+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,None,https://api.hh.ru/vacancies/48710966?host=hh.ru,https://hh.ru/vacancy/48710966,[],"{'id': '2371', 'name': 'АКРИХИН', 'url': 'http...","{'requirement': 'Знания Excel (ВПР, сводные та...",None,"{'id': 'fullDay', 'name': 'Полный день'}",{'responses': 8},[],[],[],False,noExperience
716,48415189,False,Аналитик-методолог 1С,None,False,False,"{'id': '2039', 'name': 'Люберцы', 'url': 'http...",None,"{'id': 'open', 'name': 'Открытая'}",None,None,None,2021-10-28T11:35:20+0300,2021-10-28T11:35:20+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,None,https://api.hh.ru/vacancies/48415189?host=hh.ru,https://hh.ru/vacancy/48415189,[],"{'id': '90597', 'name': 'ВОГ, Торговая компани...",{'requirement': 'Опыт <highlighttext>разработк...,None,"{'id': 'fullDay', 'name': 'Полный день'}",{'responses': 0},[],[],[],False,between1And3
89,48612930,False,Аналитик портфеля дебиторской задолженности,None,False,False,"{'id': '115', 'name': 'Киев', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}","{'city': 'Киев', 'street': 'улица Шота Руставе...",None,None,2021-10-28T14:39:25+0300,2021-10-28T14:39:25+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,None,https://api.hh.ru/vacancies/48612930?host=hh.ru,https://hh.ru/vacancy/48612930,[],"{'id': '2622530', 'name': 'TAMGA', 'url': 'htt...",{'requirement': 'Знание Excel на высоком уровн...,None,"{'id': 'fullDay', 'name': 'Полный день'}",{'responses': 5},[],[],[],False,noExperience


### Фильтрация датасета

In [4]:
# Извлечение нужных столбцов и информации из словарей в новый датафрейм

def return_id(x, key='id'):
    try:
        return x[key]
    except:
        return np.nan
    
vacancies = request_data.copy()[['id', 'name', 'published_at', 'url', 'alternate_url', 'experience']]

vacancies['employer'] = request_data['employer'].apply(return_id, key='name')
vacancies['city'] = request_data['area'].apply(return_id, key='name')
vacancies['type'] = request_data['type'].apply(return_id, key='name')
vacancies['schedule'] = request_data['schedule'].apply(return_id, key='name')
vacancies['responses'] = request_data['counters'].apply(return_id, key='responses')
vacancies['salary_from'] = request_data['salary'].apply(return_id, key='from')
vacancies['salary_to'] = request_data['salary'].apply(return_id, key='to')
vacancies['currency'] = request_data['salary'].apply(return_id, key='currency')

blackwords = ['ведущий', 'старший', 'senior', 'middle', 'teamlead', 'lead', 'руководитель', 'мидл', 'миддл', 'сеньор', 'сениор']
whitewords = ['junior', 'джуниор', 'младший', 'стажер', 'стажёр', 'интерн', 'intern']

teststr = 'саентист'
any([i in teststr.lower() for i in blackwords]) and not any([i in teststr.lower() for i in whitewords])

In [64]:
vacancies.loc[0:20, 'name']

0                                  Quantitative analyst
1                                Аналитик, Data Science
2                                        Веб - аналитик
3     Старший специалист по внедрению и сопровождени...
4                                    Системный аналитик
5                                Аналитик сопровождения
6                               Главный бизнес-аналитик
7                                  Продуктовый Аналитик
8                   Аналитик запуска сквозной аналитики
9                  Бизнес/Системный аналитик (удаленно)
10         Ведущий риск-аналитик по кредитным продуктам
11                                   Системный аналитик
12                           Data analyst/data engineer
13                                   Системный аналитик
14                           Ведущий аналитик (gamedev)
15                           Младший системный аналитик
16                                          Аналитик BI
17                                      Аналитик

In [65]:
[i in vacancies.loc[0:20, 'name'].name.lower() for i in black_words]

(vacancies['city']!='Москва')

0      False
1      False
2      False
3       True
4      False
       ...  
380    False
381    False
382    False
383    False
384    False
Name: city, Length: 385, dtype: bool

In [54]:
# Фильтрация датасета

black_words = ['ведущий', 'старший', 'senior', 'middle', 'teamlead', 'lead', 'руководитель', 
               'мидл', 'миддл', 'сеньор', 'сениор', 'тимлид', 'главный']
white_words = ['junior', 'джуниор', 'младший', 'стажер', 'стажёр', 'интерн', 'intern']

def check_name(row):
    return any([i in row['name'].str.lower() for i in black_words]) &
           (not any([i in row['name'].str.lower() for i in white_words])


mask = (
            (
                (vacancies['city']!='Москва') & 
                (vacancies['city']!='Нижний Новгород') & 
                (vacancies['schedule']!='Удаленная работа')
            ) |
    
            (
                vacancies['type']!='Открытая'
            ) 
#     |
    
#             (
#                 any([i in vacancies['name'].str.lower() for i in black_words]) &
#                 (not any([i in vacancies['name'].str.lower() for i in white_words]))
#             )
       )

vacancies.drop(labels=vacancies[mask].index, inplace=True)
vacancies.drop_duplicates(subset=['name', 'employer'], inplace=True)
vacancies.sort_values('published_at', ascending=False, inplace=True)
vacancies.reset_index(drop=True, inplace=True)

print('DataFrame shape:', request_data.shape)
vacancies.head(20)

DataFrame shape: (798, 30)


,id,name,published_at,url,alternate_url,experience,employer,city,type,schedule,responses,salary_from,salary_to,currency
0,47822315,Quantitative analyst,2021-10-29T15:13:29+0300,https://api.hh.ru/vacancies/47822315?host=hh.ru,https://hh.ru/vacancy/47822315,between1And3,Сбер Управление Активами,Москва,Открытая,Полный день,1,NaN,NaN,NaN
1,45616165,"Аналитик, Data Science",2021-10-29T15:13:18+0300,https://api.hh.ru/vacancies/45616165?host=hh.ru,https://hh.ru/vacancy/45616165,between1And3,М.Видео-Эльдорадо,Москва,Открытая,Полный день,0,NaN,NaN,NaN
2,47145862,Веб - аналитик,2021-10-29T15:13:18+0300,https://api.hh.ru/vacancies/47145862?host=hh.ru,https://hh.ru/vacancy/47145862,between1And3,М.Видео-Эльдорадо,Москва,Открытая,Полный день,0,NaN,NaN,NaN
3,40963672,Старший специалист по внедрению и сопровождени...,2021-10-29T15:07:45+0300,https://api.hh.ru/vacancies/40963672?host=hh.ru,https://hh.ru/vacancy/40963672,between1And3,Спарго Технологии,Киров (Кировская область),Открытая,Удаленная работа,0,50000.0,NaN,RUR
4,46667836,Системный аналитик,2021-10-29T14:40:37+0300,https://api.hh.ru/vacancies/46667836?host=hh.ru,https://hh.ru/vacancy/46667836,between1And3,Правительство Москвы,Москва,Открытая,Полный день,0,NaN,NaN,NaN
5,49251398,Аналитик сопровождения,2021-10-29T14:36:05+0300,https://api.hh.ru/vacancies/49251398?host=hh.ru,https://hh.ru/vacancy/49251398,noExperience,Сбербанк,Москва,Открытая,Полный день,1,NaN,NaN,NaN
6,49251114,Главный бизнес-аналитик,2021-10-29T14:29:16+0300,https://api.hh.ru/vacancies/49251114?host=hh.ru,https://hh.ru/vacancy/49251114,between1And3,МТС Банк,Москва,Открытая,Полный день,0,NaN,NaN,NaN
7,48210231,Продуктовый Аналитик,2021-10-29T14:20:02+0300,https://api.hh.ru/vacancies/48210231?host=hh.ru,https://hh.ru/vacancy/48210231,between1And3,ТД МЕДИА,Москва,Открытая,Полный день,172,250000.0,NaN,RUR
8,47481851,Аналитик запуска сквозной аналитики,2021-10-29T14:18:42+0300,https://api.hh.ru/vacancies/47481851?host=hh.ru,https://hh.ru/vacancy/47481851,between1And3,IT-Agency,Москва,Открытая,Удаленная работа,4,60000.0,120000.0,RUR
9,49250596,Бизнес/Системный аналитик (удаленно),2021-10-29T14:14:47+0300,https://api.hh.ru/vacancies/49250596?host=hh.ru,https://hh.ru/vacancy/49250596,between1And3,Eugene Global Consulting,Москва,Открытая,Удаленная работа,1,NaN,NaN,NaN


### Выгрузка и просмотр содержимого

In [6]:
# Выгрузка содержимого

vacs = vacancies.head(20)['url'].apply(lambda x: HTML(json.loads(requests.get(x).text)['description']))

In [7]:
# Настройки панели управления

items = [Button(layout=Layout(width='47.5%'), icon='arrow-left'),
         Text(value='', layout=Layout(width='5%')),
         Button(layout=Layout(width='47.5%'), icon='arrow-right')]
box_layout = Layout(display='flex', flex_flow='row', align_items='stretch', width='100%')
box = Box(children=items, layout=box_layout)
output = Output()
display(box, output)

# Функция для печати содержимого
def print_content(idx):
    clear_output()
    display(HTML('<a href="' + str(vacancies['alternate_url'][idx]) + '" target="_blank">Ссылка на вакансию</a>'))
    display(HTML("<h3>" + str(vacancies['name'][idx]) + ' в ' + str(vacancies['employer'][idx]) + "</h3>"))
    display(vacs[idx])

# Функция для реакции на клик
def on_clicked(b, direction=0):
    with output:
        try:
            global vac_idx
            try:
                vac_idx = (vac_idx + direction)%len(vacs)
            except:
                vac_idx = 0
            box.children[1].value = str(vac_idx)
            print_content(vac_idx)
        except Exception as e:
            clear_output()
            print(e)
    
# Функция для реакции на ввод текста
def on_submitted(b):
    with output:
        try:
            global vac_idx
            val = int(box.children[1].value)
            if val >= len(vacs):
                vac_idx = len(vacs) - 1
            elif val < 0:
                vac_idx = 0
            else:
                vac_idx = val
            box.children[1].value = str(vac_idx)
            print_content(vac_idx)
        except Exception as e:
            clear_output()
            print(e)

box.children[0].on_click(functools.partial(on_clicked, direction=-1))
box.children[1].on_submit(on_submitted)
box.children[2].on_click(functools.partial(on_clicked, direction=1))

Box(children=(Button(icon='arrow-left', layout=Layout(width='47.5%'), style=ButtonStyle()), Text(value='', lay…

Output()

In [8]:
#from bs4 import BeautifulSoup
#text = vacs_df['url'].apply(lambda x: BeautifulSoup(json.loads(requests.get(x).text)['description']).get_text())
#text[18]